## imagegenerator 폴더 구조 생성

- 구조
    - train
      - 1
      - 2
      - 3
      - 4
    - val
      - 1
      - 2
      - 3
      - 4
    - test
      - 1
      - 2
      - 3
      - 4

In [1]:
import random
import os
from shutil import copyfile

random.seed(a=1014) 

data_folder = '../../data/for_cnn'
image_folder = os.path.join(data_folder, 'images')
csv_file = f'{data_folder}/lon_lat_list_no_apply_is_raw.csv'
save_folder = '../../data/for_generator_korean(3_label)'
folders = [f'{save_folder}/train',f'{save_folder}/val', f'{save_folder}/test']

def make_dirs(dirs):
    if not os.path.exists(dirs):
        os.makedirs(dirs)

def get_train(data_list):
    return data_list[:int(len(data_list)*0.8)]
def get_val(data_list):
    return data_list[int(len(data_list)*0.9):]
def get_test(data_list):
    return data_list[int(len(data_list)*0.8):int(len(data_list)*0.9)]

funcdict = {
    '0': get_train,
    '1': get_val,
    '2': get_test
}



In [2]:
def process_label(dir, df, label, data_num= 10000):
    file_list = os.listdir(dir)
    df1 = df.sample(n=min(data_num, len(df)), random_state=1004, replace=False)
    for i in range(3):
        make_dirs(f'{folders[i]}\\{label}')
        data_list = funcdict[str(i)](df1['index'].values.tolist())
        for idx in data_list:
            src_path = f'{dir}/{idx}.png'
            dst_path = f'{folders[i]}/{label}/{idx}.png'
            try:
               copyfile(src_path, dst_path)
            except:
                print("null")
                
def split_4label_df(df, label):
    import numpy as np
    danger = df[df['위험도'] != 0.0]['위험도'].values.tolist()
    q1 = np.quantile(danger, 0.333333333)
    q3 = np.quantile(danger, 0.666666666)
    print(q1, q3)
    if label == 1:
        return df[df['위험도'] == 0.0]
    if label == 2:
        return df[ (df['위험도']  >0.0)  &  (df['위험도'] <= q1)]
    if label == 3:
        return df[  (df['위험도'] > q1) &  (df['위험도'] <= q3)]
    if label == 4:
        return df[df['위험도'] > q3]  

def split_3label_df(df, label):
    import numpy as np
    danger = df[df['위험도'] != 0.0]['위험도'].values.tolist()
    q1 = np.quantile(danger, 0.75)
    print(q1)
    if label == 1:
        return df[df['위험도'] == 0.0]
    if label == 2:
        return df[ (df['위험도'] >0.0)  &  (df['위험도'] <= q1)]
    if label == 3:
        return df[(df['위험도'] > q1)]

    
def process(dir, csv_path ):
    import pandas as pd
    df = pd.read_csv(csv_path, encoding='cp949', engine='python')
    for label in range(1,4):
        df2 = split_3label_df(df, label)
        print(len(df2))
        process_label(dir, df2, label)
    
            
process(image_folder, csv_file)

1.2561999999999998
10362
null
1.2561999999999998
7782
null
null
null
null
1.2561999999999998
2580
null
null
null
